In [ ]:
%glue_ray
%session_id_prefix xgboost-ray
%additional_python_modules ray[ml],xgboost,xgboost_ray

In [ ]:
%min_workers 1
%number_of_workers 2
%object_memory_worker 10

In [ ]:
import ray
from ray.train.xgboost import XGBoostTrainer
from ray.air.config import ScalingConfig
from ray.air.config import RunConfig

# Load data.
dataset = ray.data.read_csv("s3://air-example-data/breast_cancer.csv")

# Split data into train and validation.
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

trainer = XGBoostTrainer(
    scaling_config=ScalingConfig(
        # Number of workers to use for data parallelism.
        num_workers=10,
        # Whether to use GPU acceleration.
        use_gpu=False,
    ),
    run_config=RunConfig(local_dir="/tmp/ray_results"),
    label_column="target",
    num_boost_round=20,
    params={
        # XGBoost specific params
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    datasets={"train": train_dataset, "valid": valid_dataset},
)

result = trainer.fit()
print(result.metrics)

In [ ]:
%stop_session